# CS246 - Homework 1

## Question 1

### Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Get the data

In [ ]:
id='1DWp2HRuus1Ge9vvxHYLbHzyrhb_huGag'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('soc-LiveJournal1Adj.txt')

Import Spark

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd
import itertools
import re
import sys

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
dataset = sc.textFile("/content/soc-LiveJournal1Adj.txt")

In [ ]:
'''Hàm line2dataset nhận vào một chuỗi đại diện cho một cặp (src, dst_list),
tách và trả về src (kiểu int) và một danh sách các dst (kiểu list) được chuyển đổi từ chuỗi dst_line. 
Các giá trị trong danh sách dst được chuyển đổi thành kiểu int và được lọc bỏ các khoảng trắng không cần thiết.'''
def line2dataset(line):
    src, dst_line= line.split('\t')
    src = int(src.strip())
    dst_list = [int(x.strip()) for x in dst_line.split(',') if x != '']
    return src, dst_list

'''Hàm filter_pairs nhận vào một cặp dữ liệu x là một tuple chứa hai phần tử là các tuple khác: (a, [b1, b2, ..., bn]) và (c, [d1, d2, ..., dm]). 
Hàm này sẽ kiểm tra xem có thể tạo thành một cặp (a, [c, shared]) mới không, trong đó shared là số lượng phần tử chung giữa danh sách b và danh sách d.'''
def filter_pairs(x):
    if (x[0][0] != x[1][0]) and (not x[0][0] in x[1][1]) and (not x[1][0] in x[0][1]):
        shared = len(list(set(x[0][1]).intersection(set(x[1][1]))))
        return (x[0][0],[x[1][0],shared])

'''Hàm map_finaldataset nhận vào một phần tử dữ liệu elem có định dạng (src, [dst, shared]) 
và trả về một tuple mới có định dạng (src, recommendations), trong đó recommendations là danh sách 10 nút được đề xuất cho src. 
Danh sách này được tạo ra bằng cách sắp xếp các nút trong dst theo thứ tự giảm dần của số lượng nút chung shared và theo thứ tự tăng dần của giá trị dst.'''
def map_finaldataset(elem):
    src = elem[0]
    dst_commons = elem[1]
    dst_commons=sorted(dst_commons,key=lambda x:(-x[1],x[0]))[:10]
    recommendations=[pair[0] for pair in dst_commons]
    return (src, recommendations)

In [ ]:
dataset = dataset.map(line2dataset)

check_users = [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]
cartesian = dataset.cartesian(dataset).filter(lambda x: x[0][0] in check_users)

'''Dòng lệnh dưới thực hiện tính toán và tạo ra danh sách đề xuất 10 nút cho mỗi src trong RDD cartesian.'''
dataset = cartesian.map(filter_pairs).filter(lambda x: x != None and x[1][1] > 0)\
    .filter(lambda x: x[0] in check_users) \
    .groupByKey().mapValues(list).map(map_finaldataset)

id_check_dataset = dataset.filter(lambda x: x[0] in check_users).collect()

for key, val in id_check_dataset:
    print('id:', key,' recommendations:', val)

id: 9020  recommendations: [9021, 9016, 9017, 9022, 317, 9023]
id: 924  recommendations: [439, 2409, 6995, 11860, 15416, 43748, 45881]
id: 9992  recommendations: [9987, 9989, 35667, 9991]
id: 9021  recommendations: [9020, 9016, 9017, 9022, 317, 9023]
id: 9993  recommendations: [9991, 13134, 13478, 13877, 34299, 34485, 34642, 37941]
id: 8941  recommendations: [8943, 8944, 8940]
id: 9022  recommendations: [9019, 9020, 9021, 317, 9016, 9017, 9023]
id: 9990  recommendations: [13134, 13478, 13877, 34299, 34485, 34642, 37941]
id: 8942  recommendations: [8939, 8940, 8943, 8944]
id: 9019  recommendations: [9022, 317, 9023]
